In [1]:
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy

In [2]:
def parse_ingredient_list(ingredient_string):
    """Convert a string representation of a list into an actual Python list."""
    return json.loads(ingredient_string)


In [3]:
def remove_commas(text):
    """Remove all commas from the given text."""
    return text.replace(",", "")


In [4]:
df_1 = pd.read_csv('../dataset/FINAL FOOD DATASET/FOOD-DATA-GROUP1.csv')
df_2 = pd.read_csv('../dataset/FINAL FOOD DATASET/FOOD-DATA-GROUP2.csv')
df_3 = pd.read_csv('../dataset/FINAL FOOD DATASET/FOOD-DATA-GROUP3.csv')
df_4 = pd.read_csv('../dataset/FINAL FOOD DATASET/FOOD-DATA-GROUP4.csv')
df_5 = pd.read_csv('../dataset/FINAL FOOD DATASET/FOOD-DATA-GROUP5.csv')

nutrient_df = pd.concat([df_1, df_2, df_3, df_4, df_5])
nutrient_df = nutrient_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
nutrient_df

,food,Caloric Value,Fat,Saturated Fats,Monounsaturated Fats,Polyunsaturated Fats,Carbohydrates,Sugars,Protein,Dietary Fiber,...,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Zinc,Nutrition Density
0,cream cheese,51,5.000,2.900,1.300,0.200,0.8,0.500,0.900,0.000,...,0.008,14.100,0.082,0.027,1.300,0.091,15.5,19.100,0.039,7.070
1,neufchatel cheese,215,19.400,10.900,4.900,0.800,3.1,2.700,7.800,0.000,...,99.500,0.034,0.100,8.500,0.088,117.300,129.2,0.054,0.700,130.100
2,requeijao cremoso light catupiry,49,3.600,2.300,0.900,0.000,0.9,3.400,0.800,0.100,...,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,5.400
3,ricotta cheese,30,2.000,1.300,0.500,0.002,1.5,0.091,1.500,0.000,...,0.097,41.200,0.097,0.096,4.000,0.024,30.8,43.800,0.035,5.196
4,cream cheese low fat,30,2.300,1.400,0.600,0.042,1.2,0.900,1.200,0.000,...,22.200,0.072,0.008,1.200,0.098,22.800,37.1,0.034,0.053,27.007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,jews ear,25,0.095,0.000,0.000,0.000,6.7,0.000,0.500,0.000,...,15.800,0.400,0.600,24.800,0.040,13.900,42.6,0.034,0.700,24.249
718,enoki mushrooms,1,0.099,0.027,0.000,0.010,0.2,0.034,0.062,0.089,...,0.000,0.000,0.099,0.034,0.500,0.015,3.2,10.800,0.096,0.433
719,morel mushrooms,4,0.070,0.056,0.031,0.007,0.7,0.096,0.400,0.400,...,0.000,5.500,0.056,1.600,2.500,0.060,25.0,53.000,0.069,1.727
720,portabella mushrooms raw,19,0.300,0.036,0.016,0.100,3.3,2.200,1.800,1.100,...,2.600,0.200,0.300,0.000,0.092,92.900,313.0,0.015,0.500,9.400


In [5]:
nutrient_df2 = pd.read_csv('../dataset/nndb_flat.csv')
nutrient_df2 = nutrient_df2.drop(columns=['ShortDescrip', 'CommonName', 'MfgName', 'ScientificName'])
nutrient_df2['Descrip'] = nutrient_df2['Descrip'].apply(remove_commas)
nutrient_df2

,ID,FoodGroup,Descrip,Energy_kcal,Protein_g,Fat_g,Carb_g,Sugar_g,Fiber_g,VitA_mcg,...,Folate_USRDA,Niacin_USRDA,Riboflavin_USRDA,Thiamin_USRDA,Calcium_USRDA,Copper_USRDA,Magnesium_USRDA,Phosphorus_USRDA,Selenium_USRDA,Zinc_USRDA
0,1001,Dairy and Egg Products,Butter salted,717.0,0.85,81.11,0.06,0.06,0.0,684.0,...,0.0075,0.002625,0.026154,0.004167,0.020000,0.000000,0.004762,0.034286,0.018182,0.008182
1,1002,Dairy and Egg Products,Butter whipped with salt,717.0,0.85,81.11,0.06,0.06,0.0,684.0,...,0.0075,0.002625,0.026154,0.004167,0.020000,0.000018,0.004762,0.032857,0.018182,0.004545
2,1003,Dairy and Egg Products,Butter oil anhydrous,876.0,0.28,99.48,0.00,0.00,0.0,840.0,...,0.0000,0.000188,0.003846,0.000833,0.003333,0.000001,0.000000,0.004286,0.000000,0.000909
3,1004,Dairy and Egg Products,Cheese blue,353.0,21.40,28.74,2.34,0.50,0.0,198.0,...,0.0900,0.063500,0.293846,0.024167,0.440000,0.000044,0.054762,0.552857,0.263636,0.241818
4,1005,Dairy and Egg Products,Cheese brick,371.0,23.24,29.68,2.79,0.51,0.0,292.0,...,0.0500,0.007375,0.270000,0.011667,0.561667,0.000027,0.057143,0.644286,0.263636,0.236364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8613,83110,Finfish and Shellfish Products,Fish mackerel salted,305.0,18.50,25.10,0.00,0.00,0.0,47.0,...,0.0375,0.206250,0.146154,0.016667,0.055000,0.000111,0.142857,0.362857,1.334545,0.100000
8614,90240,Finfish and Shellfish Products,Mollusks scallop (bay and sea) cooked steamed,111.0,20.54,0.84,5.41,0.00,0.0,2.0,...,0.0500,0.067250,0.018462,0.010000,0.008333,0.000037,0.088095,0.608571,0.394545,0.140909
8615,90480,Sweets,Syrup Cane,269.0,0.00,0.00,73.14,73.20,0.0,0.0,...,0.0000,0.006250,0.046154,0.108333,0.010833,0.000022,0.023810,0.011429,0.012727,0.017273
8616,90560,Finfish and Shellfish Products,Mollusks snail raw,90.0,16.10,1.40,2.00,0.00,0.0,30.0,...,0.0150,0.087500,0.092308,0.008333,0.008333,0.000444,0.595238,0.388571,0.498182,0.090909


In [6]:
# Mapping from original column names to the ones in nutritional_info
column_mapping_1 = {
    'food': 'Ingredients',
    'Caloric Value': 'Calories',
    'Fat': 'Fat',
    'Protein': 'Protein',
    'Carbohydrates': 'Carb',
    'Calcium': 'Calcium',
    'Sugars': 'Sugar',
    'Dietary Fiber': 'Fiber',
    'Vitamin A': 'VitA',
    'Vitamin B6': 'VitB6',
    'Vitamin B12': 'VitB12',
    'Vitamin C': 'VitC',
    'Vitamin E': 'VitE',
}

# Select only relevant columns and rename them
nutrient_df_filtered = nutrient_df[list(column_mapping_1.keys())].rename(columns=column_mapping_1)
nutrient_df_filtered['Source'] = 1

In [7]:
# Mapping from original column names to the ones in nutritional_info format
column_mapping_2 = {
    'Descrip': 'Ingredients',
    'Energy_kcal': 'Calories',
    'Fat_g': 'Fat',
    'Protein_g': 'Protein',
    'Carb_g': 'Carb',
    'Calcium_mg': 'Calcium',
    'Sugar_g': 'Sugar',
    'Fiber_g': 'Fiber',
    'VitA_mcg': 'VitA',
    'VitB6_mg': 'VitB6',
    'VitB12_mcg': 'VitB12',
    'VitC_mg': 'VitC',
    'VitE_mg': 'VitE'
}

# Select only relevant columns and rename them
nutrient_df2_filtered = nutrient_df2[list(column_mapping_2.keys())].rename(columns=column_mapping_2)
nutrient_df2_filtered['Source'] = 2

In [ ]:
combined_df = pd.concat([nutrient_df_filtered, nutrient_df2_filtered])
combined_df.shape

(11013, 14)

In [9]:
def handle_duplicates(df, threshold=0.8):
    """
    Detects and handles duplicate ingredients from different sources using TF-IDF and cosine similarity.
    Retains only ingredients from Source = 2 when duplicates are found.
    Logs the detected duplicates and the kept ingredient.
    
    Parameters:
        df (pd.DataFrame): The input dataframe containing 'Ingredients' and 'Source' columns.
        threshold (float): The cosine similarity threshold for detecting duplicates (default is 0.85).

    Returns:
        pd.DataFrame: A cleaned dataframe with duplicates handled.
    """
    df = df.copy()
    
    # Separate by source
    df1 = df[df['Source'] == 1].reset_index(drop=True)
    df2 = df[df['Source'] == 2].reset_index(drop=True)
    
    # Vectorize ingredient names
    vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2,4))
    tfidf_matrix_1 = vectorizer.fit_transform(df1['Ingredients'])
    tfidf_matrix_2 = vectorizer.transform(df2['Ingredients'])
    
    # Compute cosine similarity between source 1 and source 2
    similarity_matrix = cosine_similarity(tfidf_matrix_1, tfidf_matrix_2)
    
    # Identify duplicates
    duplicate_log = []
    to_remove = set()
    
    for i, row_sim in enumerate(similarity_matrix):
        max_sim_idx = np.argmax(row_sim)
        max_sim_value = row_sim[max_sim_idx]
        
        if max_sim_value >= threshold:
            dup_ingredient = df1.loc[i, 'Ingredients']
            keep_ingredient = df2.loc[max_sim_idx, 'Ingredients']
            
            duplicate_log.append(f"Duplicate found: '{dup_ingredient}' (Source 1) -> Keeping '{keep_ingredient}' (Source 2)")
            to_remove.add(df1.index[i])
    
    # Remove duplicates from source 1
    df_cleaned = df.drop(index=to_remove).reset_index(drop=True)
    
    # Log detected duplicates
    print("Duplicate Log:")
    for log in duplicate_log:
        print(log)
    
    # Log total changes
    print(f"Total duplicates removed: {len(to_remove)}")
    
    return df_cleaned

combined_df = handle_duplicates(combined_df)

Duplicate Log:
Duplicate found: 'cream cheese' (Source 1) -> Keeping 'Cheese cream' (Source 2)
Duplicate found: 'neufchatel cheese' (Source 1) -> Keeping 'Cheese neufchatel' (Source 2)
Duplicate found: 'cream cheese low fat' (Source 1) -> Keeping 'Cheese cream low fat' (Source 2)
Duplicate found: 'cream cheese fat free' (Source 1) -> Keeping 'Cheese cream fat free' (Source 2)
Duplicate found: 'gruyere cheese' (Source 1) -> Keeping 'Cheese gruyere' (Source 2)
Duplicate found: 'cheddar cheese' (Source 1) -> Keeping 'Cheese cheddar' (Source 2)
Duplicate found: 'parmesan cheese' (Source 1) -> Keeping 'Cheese parmesan hard' (Source 2)
Duplicate found: 'romano cheese' (Source 1) -> Keeping 'Cheese romano' (Source 2)
Duplicate found: 'parmesan cheese grated' (Source 1) -> Keeping 'Cheese parmesan grated' (Source 2)
Duplicate found: 'port salut cheese' (Source 1) -> Keeping 'Cheese port de salut' (Source 2)
Duplicate found: 'swiss cheese' (Source 1) -> Keeping 'Cheese swiss' (Source 2)
Duplica

In [10]:
combined_df.shape

(9157, 14)

In [14]:
combined_df.to_csv('combined.csv')

In [11]:
resep_df = pd.read_csv('../dataset/dataset-resep/recipes_sample.csv')
resep_df = resep_df.drop(columns=['link', 'source', 'site'])
resep_df['ingredients'] = resep_df['ingredients'].apply(parse_ingredient_list)
resep_df['directions'] = resep_df['directions'].apply(parse_ingredient_list)
resep_df['NER'] = resep_df['NER'].apply(parse_ingredient_list)
resep_df

,title,ingredients,directions,NER
0,Marinated Flank Steak Recipe,"[1 1/2 pound flank steak, 1/2 c. finely minced...","[Remove tenderloin from steak., Score meat., C...","[soy sauce, green onions, salad oil, sesame se..."
1,French Chicken Stew,"[1 tablespoon rosemary, 1 teaspoon thyme, 3 ba...",[combine all ingredients in slow cooker (6 qua...,"[mushroom mix, pepper, button mushrooms, stalk..."
2,Glazed Carrots,"[3 to 4 carrots, 1 1/2 Tbsp. butter, 1/3 c. br...",[Cook 3 to 4 carrots; cut crosswise in 1-inch ...,"[butter, brown sugar, carrots, lemon rind]"
3,Moms Pie Dough,"[4.5 Cups Flour, 1.5 Tsp Salt, Pinch Baking Po...","[Mix all dry ingredients in a bowl., , Add cri...","[egg, baking powder, sugar, crisco, vinegar, w..."
4,Pretzel Salad Or Dessert,"[2 c. crushed small thin pretzels (sticks), 3/...","[Mix and press in baking pan, approximately 13...","[thin pretzels, margarine]"
...,...,...,...,...
9995,Bircher Muesli (Swiss Oatmeal),"[3 cups rolled oats, 3/4 cup fresh orange juic...","[In a large bowl, combine the rolled oats, ora...","[apple, sugar, ground nutmeg, sultana, vanilla..."
9996,Tyler Texas Salad,"[4 summer squash, 2 cups orange marmalade, 1 c...","[Slice squash into 1/4 inch""wheels,"" and toss ...","[poppy seeds, sugar, shredded coconut, black p..."
9997,Western Hash,[1 lb. ground beef Safeway 1 lb For $3.99 thru...,[Brown meat in large skillet on medium-high he...,"[tomatoes, long-grain white rice, onion, groun..."
9998,Holiday Mash Potatoes,[2 medium butternut squash baked and cut into ...,[Place potatoes in large pan and cover with wa...,"[fingerling potatoes, sour cream, white peoppe..."


In [20]:
resep_df.to_csv('resep.csv')

In [13]:
# Load spaCy's language model with word vectors
nlp = spacy.load("en_core_web_md")  # Use 'en_core_web_md' or 'en_core_web_lg' for better results

def spacy_match_ingredients(ingredients, reference_df, ingredient_col="Descrip", calorie_col="Calories", threshold=0.5):
    """
    Match each ingredient in 'ingredients' to the closest match in 'reference_df'
    using spaCy NLP similarity scoring, and retrieve its caloric value.

    Args:
        ingredients (list): List of ingredients to match.
        reference_df (pd.DataFrame): DataFrame containing reference ingredients and their caloric values.
        ingredient_col (str): Column name for ingredient descriptions in reference_df.
        calorie_col (str): Column name for caloric values in reference_df.
        threshold (float): Minimum similarity score (0-1) to consider a valid match.

    Returns:
        dict: A dictionary mapping each ingredient to a tuple (best match, caloric value) or (None, None).
    """
    reference_ingredients = reference_df[ingredient_col].astype(str).tolist()
    reference_docs = [nlp(ing) for ing in reference_ingredients]  # Precompute vector representations

    matched_ingredients = {}
    for ingredient in ingredients:
        ingredient_doc = nlp(ingredient)
        similarities = np.array([ingredient_doc.similarity(ref_doc) for ref_doc in reference_docs])
        best_match_idx = np.argmax(similarities)
        best_match_score = similarities[best_match_idx]

        if best_match_score >= threshold:
            best_match = reference_df.iloc[best_match_idx]
            matched_ingredients[ingredient] = (best_match[ingredient_col], best_match[calorie_col])
        else:
            matched_ingredients[ingredient] = (None, None)

    return matched_ingredients

# Example usage
matched_results = spacy_match_ingredients(resep_df['NER'][100], combined_df, ingredient_col="Ingredients", calorie_col="Calories")

# Print results
for ingredient, (match, calories) in matched_results.items():
    print(f"{ingredient} -> {match} (Calories: {calories})")


C:\Users\ASUS\AppData\Local\Temp\ipykernel_12088\1901809424.py:25: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarities = np.array([ingredient_doc.similarity(ref_doc) for ref_doc in reference_docs])


ground pork -> ground pork cooked (Calories: 930.0)
cabbage -> savoy cabbage cooked (Calories: 41.0)
pepper -> longans dried (Calories: 5.0)
lemon juice -> tangerine juice (Calories: 106.0)
brown sugar -> brown sugar (Calories: 16.0)
onion -> mustard (Calories: 3.0)
rice -> coconut whole coconut (Calories: 3336.0)
ground beef -> ground beef cooked (Calories: 375.0)
tomato sauce -> tomato soup (Calories: 98.0)
salt -> longans dried (Calories: 5.0)


In [14]:
def tfidf_match_ingredients(ingredients, reference_df, ingredient_col="Descrip", calorie_col="Calories", threshold=0.5):
    """
    Match each ingredient in 'ingredients' to the closest match in 'reference_df'
    using TF-IDF and cosine similarity, and retrieve its caloric value.

    Args:
        ingredients (list): List of ingredients to match.
        reference_df (pd.DataFrame): DataFrame containing reference ingredients and their caloric values.
        ingredient_col (str): Column name for ingredient descriptions in reference_df.
        calorie_col (str): Column name for caloric values in reference_df.
        threshold (float): Minimum cosine similarity (0-1) to consider a valid match.

    Returns:
        dict: A dictionary mapping each ingredient to a tuple (best match, caloric value) or (None, None).
    """
    reference_ingredients = reference_df[ingredient_col].astype(str).tolist()
    vectorizer = TfidfVectorizer().fit(reference_ingredients + ingredients)
    reference_vectors = vectorizer.transform(reference_ingredients)

    matched_ingredients = {}
    for ingredient in ingredients:
        ingredient_vector = vectorizer.transform([ingredient])
        similarities = cosine_similarity(ingredient_vector, reference_vectors)[0]
        best_match_idx = np.argmax(similarities)
        best_match_score = similarities[best_match_idx]

        if best_match_score >= threshold:
            best_match = reference_df.iloc[best_match_idx]
            matched_ingredients[ingredient] = (best_match[ingredient_col], best_match[calorie_col])
        else:
            matched_ingredients[ingredient] = (None, None)

    return matched_ingredients

# Example usage
matched_results = tfidf_match_ingredients(resep_df['NER'][100], combined_df, ingredient_col="Ingredients", calorie_col="Calories")

# Print results
for ingredient, (match, calories) in matched_results.items():
    print(f"{ingredient} -> {match} (Calories: {calories})")


ground pork -> ground pork cooked (Calories: 930.0)
cabbage -> cabbage raw (Calories: 70.0)
pepper -> white pepper (Calories: 7.0)
lemon juice -> lemon juice (Calories: 54.0)
brown sugar -> brown sugar (Calories: 16.0)
onion -> onion raw (Calories: 16.0)
rice -> fried rice (Calories: 228.0)
ground beef -> ground beef cooked (Calories: 375.0)
tomato sauce -> tomato chili sauce (Calories: 6.0)
salt -> salt pork raw (Calories: 212.0)


In [17]:
resep_df['NER'][0]

['soy sauce',
 'green onions',
 'salad oil',
 'sesame seeds',
 'brown sugar',
 'flank steak',
 'clove garlic',
 'grnd ginger',
 'grnd black pepper',
 'red wine']